In [1]:
import pandas as pd
import numpy as np
import peewee
from playhouse import postgres_ext
from data_model import *
import json
import random
from pprint import pprint

from mmr import mmr
from sklearn import metrics,preprocessing
from scipy.spatial import distance

/Users/User/.virtualenvs/window_study/lib/python3.7/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [2]:
features = []
for i in TaskType.select():
    print(i.task_keywords)
    features.extend(i.task_keywords)
features = list(set(features))

['sentiment analysis', 'text categorization', 'climate change']
['word comparison', 'semantic similarity']
['sentence agreement', 'sentence comparison']
['image annotation', 'image rating']
['image categorization', 'sentiment analysis']
['image categorization', 'pattern recognition', 'image annotation']
['transcription', 'image categorization', 'handwriting']
['semantic analysis', 'semantic similarity', 'comparison']
['sentiment analysis', 'emotion detection', 'resolutions']
['sentiment analysis', 'emotion detection', 'self-driving cars']


In [3]:
features

['climate change',
 'image categorization',
 'sentence agreement',
 'text categorization',
 'self-driving cars',
 'image rating',
 'pattern recognition',
 'emotion detection',
 'resolutions',
 'sentence comparison',
 'semantic similarity',
 'semantic analysis',
 'comparison',
 'word comparison',
 'handwriting',
 'sentiment analysis',
 'transcription',
 'image annotation']

In [4]:
workers = pd.read_csv('data/20190306-User Profiles.csv',encoding='latin')

In [5]:
worker_keywords = pd.DataFrame(0,columns=features,index=workers.WorkerId)

In [6]:
for i,wk in workers.loc[:,['Keywords','WorkerId']].iterrows():
    for k in wk.Keywords.split('|'):
        if k in features:
            worker_keywords.loc[wk.WorkerId,k] = 1
        

In [7]:
task_type_features = pd.DataFrame(0,columns=features,index=range(1,11))
for t in TaskType.select():
    i = t.task_type_id
    for d in t.task_keywords:
        task_type_features.loc[i,d] = 1


In [8]:
class sim_rel:
    def __init__(self,worker_id,task_ids):
        self.worker_id = worker_id
        self.task_worker_dist_matrix = self.generate_normalized_sim(task_ids)
        self.norm_tasks =  self.generate_normalized_tasks()
        
    def generate_normalized_sim(self,task_ids):
        task_worker_dist_matrix = []
        for i in task_ids:
            task_worker_dist_matrix.append(metrics.pairwise.cosine_similarity(worker_keywords.loc[self.worker_id,:].values.reshape(1,-1),task_type_features.loc[i,:].values.reshape(1,-1)))
        task_worker_dist_matrix = np.array(task_worker_dist_matrix).reshape(-1,1)
        task_worker_dist_matrix = preprocessing.StandardScaler().fit_transform(task_worker_dist_matrix)
        task_worker_dist_matrix = list(task_worker_dist_matrix)
        return dict(zip(task_ids,task_worker_dist_matrix))

    def generate_normalized_tasks(self):
        norm_pay = []
        task_ids = []
        for t in TaskType.select():
            norm_pay.append(t.expected_pay)
            task_ids.append(t.task_type_id)
        norm_pay = np.array(norm_pay).reshape(-1,1)
        norm_pay = preprocessing.StandardScaler().fit_transform(norm_pay)
        return dict(zip(task_ids,norm_pay))

    def sim_rel(self,task_id,worker_id=None):
        if task_id in self.task_worker_dist_matrix :             
            return self.task_worker_dist_matrix[task_id]

    def sim_payment(self,tid_1,tid_2):
        return (self.norm_tasks[tid_1] - self.norm_tasks[tid_2])**2

In [9]:
### RANDOM
for worker in worker_keywords.index:
    print(worker)
    tasks = [str(t.id) for t in Task.select().order_by(fn.Random()).limit(9)]
    with open("./data/assignments/random/{0}.tasks".format(worker),'w') as f:
        f.write("#{0}".format(worker))
        f.write("\n")
        f.write("{")
        f.write(",".join(tasks[0:3]))
        f.write("}")
        f.write("\n")
        f.write("{")
        f.write(",".join(tasks[3:6]))
        f.write("}")
        f.write("\n")
        f.write("{")
        f.write(",".join(tasks[6:]))
        f.write("}")

A36X6B6HXD442I
A3VEOKRNA12U0E
A1FN0TLGQJOR3O
ARN81BLM2EIF2
A30NX5HEQGGFPP
A2C7EYRPWL8B3L
A2VDBWIHWE1SVD
A1ZGLO0466I754
A1QUOTZEN3BNVF
A1QO8VYG8C4J
A1EZ6HN22MCUUG
A1J9CJOLZRVLV
A1C7XI68SED8JE
A2MWBFGSAZMBUR
AAB9L29X52ZIY
A2I988BSR2YYU3
A3FTR5MN7ZLS8M
AKDELYTK8CTJ2
AY7RI824FORAE
A2741GQBQ2JSDU
A247PDSKLZU86P
AIDGP0N4U40ME
A35VANDENR32UZ
A2PTPH3BC7LX4O
A3F99X0TW7OCLP
A1PTC5FHUYMWYL
AHYXP6QIMYJLD
A31YAHLCTIQNFL
A3FKX13NFQZP0Q
ACG4WOR7KO0Y
A1QXQBZE7ROQ4F
AV182SXSO370E
A3B77HFHIPFZB6
A2YQ2RNOK49TY0
A3N17E39KZOQX0
A2A6CBJZZDJNWF
A3100E4TIT3L9L
AWW7U5TW7NPKO
A16QJWRA538JC
A2X00K0L67M360
A1KXG7W711GZDJ
A1DK26QAO4OOMQ
AWS49CZW5Y9JK
AGN2WNWU7JSYF
A2LBFHF761B64Y
A1AEJ03AZ388NS
AMA18W8F60Y2J
A11UG30LL53IHA
A2PJBSDDN22XIX
A1G3BVCKXMEUPU
A10U8R4IEOHQUA
A3MU0ATDYGXJR0
A25AYALHIJRNKR
AEOFJWOEXF94U
A12MSYRHI3XWDT
A1QRXIMSBSK6YR
A1XCR8UPNMH8L
A25823ZSJA9TCV
A1SFBJ5WW2OZTI
ANWR7DTA3OLLN
A37AJI03M37NPJ
A8289AEOKHKV4
A2KHTVC22XK64X
A3SIYZQYIHULPB
A128VDS8TMHBDP
A1LEWJHT00OKV9
A1E3FZULK1PTWB
AJUSYY64UFMI5
AM

In [113]:
sim_func = sim_rel("A317RA15E0SZ4P",list(range(1,11)))

In [10]:
# MMR
for w in sorted(worker_keywords.index):
    print(w)
    sim_func = sim_rel(w,list(range(1,11)))
    x_good,_ = mmr(3,list(range(1,11)),w,0.7,0.3,sim_func.sim_rel,sim_func.sim_payment)
    pprint(x_good)
    task_1 = [str(t.id) for t in Task.select().where(Task.task_type.in_(x_good)).order_by(fn.Random()).limit(9)]
    print(task_1)
    with open("./data/assignments/mmr/{0}.tasks".format(w),'w') as f:
        f.write("#{0}".format(w))
        f.write("\n")
        f.write("{")
        f.write(",".join(task_1[0:3]))
        f.write("}")
        f.write("\n")
        f.write("{")
        f.write(",".join(task_1[3:6]))
        f.write("}")
        f.write("\n")
        f.write("{")
        f.write(",".join(task_1[6:]))
        f.write("}")
    

A10U8R4IEOHQUA
[6, 4, 2]
['6289', '7093', '3662', '3039', '7853', '10513', '6215', '10015', '7974']
A11LSO6D7BMY99
[4, 7, 1]
['13855', '13285', '6321', '1866', '6059', '13531', '13526', '14000', '13717']
A11UG30LL53IHA
[1, 10, 5]
['9873', '403', '327', '19932', '432', '9922', '586', '171', '18402']
A128VDS8TMHBDP
[4, 6, 9]
['17033', '16774', '10380', '10148', '16503', '7073', '7155', '16116', '10839']
A12MSYRHI3XWDT
[6, 4, 9]
['17860', '11561', '7417', '16961', '7534', '6783', '10434', '11507', '16507']
A12O9P1OTMJOPW
[1, 10, 5]
['9146', '18682', '8572', '18841', '853', '18914', '18929', '9779', '19214']
A153IWW5M88F18
[4, 9, 6]
['10059', '16333', '16509', '6391', '6009', '7649', '11195', '11276', '17741']
A15C1I7BG3FBG4
[6, 4, 7]
['12936', '12150', '7814', '12106', '13485', '13477', '7222', '6489', '11161']
A15G28BPBQLE76
[1, 10, 5]
['1509', '1328', '9068', '9265', '18148', '19753', '19302', '9534', '19210']
A16QJWRA538JC
[2, 7, 10]
['13757', '2057', '2832', '18872', '19112', '2843', 

['11643', '6254', '5266', '6749', '11825', '10686', '6177', '6283', '6827']
A2IU5MWCG7GQXH
[4, 6, 9]
['6465', '7847', '7099', '16446', '17034', '6793', '6736', '17120', '10131']
A2JTV44ZIGMS5D
[2, 5, 7]
['8881', '2275', '12268', '9844', '13536', '2424', '3690', '9698', '2041']
A2KHTVC22XK64X
[1, 7, 5]
['12252', '9707', '1930', '12225', '610', '9434', '12132', '390', '148']
A2KKUKAW8EWDAO
[3, 7, 10]
['5222', '19958', '5455', '18551', '18087', '18422', '18760', '18173', '19596']
A2LAE3OM5OQ0WF
[1, 10, 7]
['19054', '19001', '13264', '18400', '1156', '13694', '1254', '162', '1711']
A2LBFHF761B64Y
[4, 6, 9]
['11134', '11066', '10375', '17064', '10679', '11574', '6726', '11048', '10855']
A2M0II20JK7R13
[4, 6, 9]
['11959', '10265', '17460', '16876', '10456', '17034', '7900', '16895', '6545']
A2M4QB5T835AW1
[1, 5, 10]
['8727', '18542', '1395', '1608', '18193', '8435', '9225', '333', '8148']
A2MD0PWLTAHVW4
[1, 5, 10]
['769', '1774', '8297', '1990', '9215', '19276', '19404', '933', '8206']
A2MMT

[2, 1, 7]
['215', '13789', '12036', '13510', '604', '13892', '13881', '672', '12990']
AHYXP6QIMYJLD
[6, 4, 9]
['10521', '11187', '11669', '6493', '16322', '6252', '10338', '7180', '6583']
AIDGP0N4U40ME
[1, 10, 7]
['689', '1726', '902', '18077', '18911', '13362', '18415', '129', '19203']
AJUSYY64UFMI5
[9, 2, 3]
['5169', '3925', '17244', '4080', '4084', '4744', '16882', '2948', '3056']
AJVZEONGPDAEV
[1, 10, 5]
['1439', '8854', '8506', '19329', '8308', '9115', '18974', '1101', '18526']
AKDELYTK8CTJ2
[9, 6, 4]
['11500', '11833', '11873', '10888', '7688', '11758', '6391', '10211', '6942']
AKJTYKU7Y3CSL
[1, 5, 10]
['522', '19703', '19689', '8029', '18843', '18767', '188', '9596', '19167']
AKUKBGCEJGSDA
[2, 7, 1]
['806', '13461', '3862', '12650', '13496', '210', '1458', '12048', '113']
AL18PX0MPLANX
[1, 5, 10]
['1089', '9879', '8726', '9187', '8232', '1069', '19615', '18895', '19690']
AMA18W8F60Y2J
[5, 1, 10]
['8535', '8802', '474', '30', '351', '9685', '8646', '9676', '545']
AMEBYQ08FA7H8
[1

([1, 5, 10],
 [(1, array([1.00809146])),
  (5, array([0.25121965])),
  (10, array([0.03180406]))])

In [11]:
for w in sorted(worker_keywords.index):
    print(w)
    x = sorted({(t,TaskType.select().where(TaskType.task_type_id==t).get().requester):distance.cosine(worker_keywords.loc[w,:].values,task_type_features.loc[t,:].values) for t in range(1,11)}.items(),key=lambda x : x[1])
    x_good = [xx for xx in x if xx[1] < 0.7]
    pprint(x_good)
    x_tasks = [i[0][0] for i in x_good]
    pprint(x_tasks)
    tasks = [str(t.id) for t in Task.select().where(Task.task_type.in_(x_tasks)).order_by(fn.Random()).limit(9)]
    with open("./data/assignments/ct/{0}.tasks".format(w),'w') as f:
        f.write("#{0}".format(w))
        f.write("\n")
        f.write("{")
        f.write(",".join(tasks[0:3]))
        f.write("}")
        f.write("\n")
        f.write("{")
        f.write(",".join(tasks[3:6]))
        f.write("}")
        f.write("\n")
        f.write("{")
        f.write(",".join(tasks[6:]))
        f.write("}")
    

A10U8R4IEOHQUA
[((6, 'req2'), 0.42264973081037427),
 ((4, 'req1'), 0.5285954792089683),
 ((1, 'req4'), 0.6150998205402495),
 ((7, 'req1'), 0.6150998205402495)]
[6, 4, 1, 7]
A11LSO6D7BMY99
[((4, 'req1'), 0.6464466094067263)]
[4]
A11UG30LL53IHA
[((1, 'req4'), 0.5527864045000421),
 ((6, 'req2'), 0.5527864045000421),
 ((9, 'req2'), 0.5527864045000421),
 ((10, 'req1'), 0.5527864045000421),
 ((2, 'req1'), 0.6348516283298893),
 ((3, 'req3'), 0.6348516283298893),
 ((4, 'req1'), 0.6348516283298893),
 ((5, 'req2'), 0.6348516283298893)]
[1, 6, 9, 10, 2, 3, 4, 5]
A128VDS8TMHBDP
[((4, 'req1'), 0.36754446796632423),
 ((6, 'req2'), 0.4836022205056777),
 ((10, 'req1'), 0.4836022205056777),
 ((5, 'req2'), 0.6837722339831621)]
[4, 6, 10, 5]
A12MSYRHI3XWDT
[((6, 'req2'), 0.4777670321329065),
 ((4, 'req1'), 0.5735985672887791),
 ((1, 'req4'), 0.6518446880886044),
 ((7, 'req1'), 0.6518446880886044),
 ((9, 'req2'), 0.6518446880886044),
 ((10, 'req1'), 0.6518446880886044)]
[6, 4, 1, 7, 9, 10]
A12O9P1OTMJOPW


A1SFBJ5WW2OZTI
[((4, 'req1'), 0.5),
 ((9, 'req2'), 0.591751709536137),
 ((10, 'req1'), 0.591751709536137)]
[4, 9, 10]
A1U369G5MSDPZ2
[((7, 'req1'), 0.5635642195280153)]
[7]
A1U4ORTCCDLLW9
[((6, 'req2'), 0.5),
 ((7, 'req1'), 0.5),
 ((4, 'req1'), 0.591751709536137),
 ((1, 'req4'), 0.6666666666666667),
 ((9, 'req2'), 0.6666666666666667),
 ((10, 'req1'), 0.6666666666666667)]
[6, 7, 4, 1, 9, 10]
A1W9WQEZNFWA0P
[((2, 'req1'), 0.5285954792089683),
 ((1, 'req4'), 0.6150998205402495),
 ((8, 'req3'), 0.6150998205402495),
 ((10, 'req1'), 0.6150998205402495)]
[2, 1, 8, 10]
A1XCR8UPNMH8L
[((2, 'req1'), 0.6464466094067263)]
[2]
A1YNBQAKX6FUVW
[((9, 'req2'), 0.42264973081037427),
 ((10, 'req1'), 0.42264973081037427),
 ((5, 'req2'), 0.5285954792089683),
 ((1, 'req4'), 0.6150998205402495),
 ((7, 'req1'), 0.6150998205402495)]
[9, 10, 5, 1, 7]
A1Z0VVDFYH9PBW
[((10, 'req1'), 0.4836022205056777),
 ((4, 'req1'), 0.6837722339831621),
 ((5, 'req2'), 0.6837722339831621)]
[10, 4, 5]
A1Z4I9V3FSMKZ3
[((6, 'req2')

[((2, 'req1'), 0.591751709536137),
 ((7, 'req1'), 0.6666666666666667),
 ((9, 'req2'), 0.6666666666666667),
 ((10, 'req1'), 0.6666666666666667)]
[2, 7, 9, 10]
A2MWBFGSAZMBUR
[((1, 'req4'), 0.6150998205402495),
 ((9, 'req2'), 0.6150998205402495),
 ((10, 'req1'), 0.6150998205402495)]
[1, 9, 10]
A2O7YJDVLPCWOK
[((2, 'req1'), 0.4654775161751513)]
[2]
A2P1PQRINHULV7
[((3, 'req3'), 0.42264973081037427), ((1, 'req4'), 0.5285954792089683)]
[3, 1]
A2P2RAA158TDJY
[((1, 'req4'), 0.5527864045000421),
 ((6, 'req2'), 0.5527864045000421),
 ((7, 'req1'), 0.5527864045000421),
 ((9, 'req2'), 0.5527864045000421),
 ((10, 'req1'), 0.5527864045000421),
 ((4, 'req1'), 0.6348516283298893),
 ((5, 'req2'), 0.6348516283298893)]
[1, 6, 7, 9, 10, 4, 5]
A2PJBSDDN22XIX
[((4, 'req1'), 0.2928932188134524), ((2, 'req1'), 0.6464466094067263)]
[4, 2]
A2PTPH3BC7LX4O
[((10, 'req1'), 0.4836022205056777),
 ((2, 'req1'), 0.6837722339831621),
 ((3, 'req3'), 0.6837722339831621),
 ((5, 'req2'), 0.6837722339831621)]
[10, 2, 3, 5]


A3KM6AF7CGASHJ
[((6, 'req2'), 0.5370899501137243),
 ((2, 'req1'), 0.6220355269907728),
 ((3, 'req3'), 0.6220355269907728),
 ((4, 'req1'), 0.6220355269907728),
 ((1, 'req4'), 0.6913933000758161),
 ((7, 'req1'), 0.6913933000758161),
 ((8, 'req3'), 0.6913933000758161),
 ((10, 'req1'), 0.6913933000758161)]
[6, 2, 3, 4, 1, 7, 8, 10]
A3MN8AL9A0BJVZ
[((4, 'req1'), 0.6464466094067263)]
[4]
A3MO5U0K4J64RR
[((4, 'req1'), 0.5),
 ((5, 'req2'), 0.5),
 ((1, 'req4'), 0.591751709536137),
 ((6, 'req2'), 0.591751709536137),
 ((7, 'req1'), 0.591751709536137),
 ((9, 'req2'), 0.591751709536137),
 ((10, 'req1'), 0.591751709536137)]
[4, 5, 1, 6, 7, 9, 10]
A3MU0ATDYGXJR0
[((9, 'req2'), 0.591751709536137), ((10, 'req1'), 0.591751709536137)]
[9, 10]
A3N17E39KZOQX0
[((9, 'req2'), 0.29289321881345254),
 ((10, 'req1'), 0.29289321881345254),
 ((5, 'req2'), 0.42264973081037427),
 ((1, 'req4'), 0.5285954792089683)]
[9, 10, 5, 1]
A3NEAC780MRHAN
[((4, 'req1'), 0.6464466094067263)]
[4]
A3P24DLGZPK4WM
[((6, 'req2'), 0.48

[((6, 'req2'), 0.4777670321329065),
 ((4, 'req1'), 0.5735985672887791),
 ((7, 'req1'), 0.6518446880886044),
 ((8, 'req3'), 0.6518446880886044),
 ((9, 'req2'), 0.6518446880886044),
 ((10, 'req1'), 0.6518446880886044)]
[6, 4, 7, 8, 9, 10]
AX5CQBYN2TM72
[((1, 'req4'), 0.5799159747915971),
 ((6, 'req2'), 0.5799159747915971),
 ((7, 'req1'), 0.5799159747915971),
 ((9, 'req2'), 0.5799159747915971),
 ((10, 'req1'), 0.5799159747915971),
 ((2, 'req1'), 0.6570028297149824),
 ((3, 'req3'), 0.6570028297149824),
 ((4, 'req1'), 0.6570028297149824),
 ((5, 'req2'), 0.6570028297149824)]
[1, 6, 7, 9, 10, 2, 3, 4, 5]
AX8P6X022Q64
[((1, 'req4'), 0.5370899501137243),
 ((7, 'req1'), 0.5370899501137243),
 ((9, 'req2'), 0.5370899501137243),
 ((2, 'req1'), 0.6220355269907728),
 ((3, 'req3'), 0.6220355269907728),
 ((5, 'req2'), 0.6220355269907728),
 ((6, 'req2'), 0.6913933000758161),
 ((10, 'req1'), 0.6913933000758161)]
[1, 7, 9, 2, 3, 5, 6, 10]
AXTFSJDN2YLO5
[((9, 'req2'), 0.5),
 ((10, 'req1'), 0.5),
 ((2, 'req

In [12]:

for w in sorted(worker_keywords.index):
    print(w)
    x = sorted({(t,TaskType.select().where(TaskType.task_type_id==t).get().expected_pay):distance.cosine(worker_keywords.loc[w,:].values,task_type_features.loc[t,:].values) for t in range(1,11)}.items(),key=lambda x : x[1])
    x_good = [xx for xx in x if xx[1] < 0.4]
    if len(x_good) < 1:
        x_good = [xx for xx in x if xx[1] < 0.6]
    if len(x_good) < 1:
        x_good = [xx for xx in x if xx[1] < 0.7]
    if len(x_good) < 1:
        x_good = [xx for xx in x if xx[1] < 0.8]
    if len(x_good) < 1:
        x_good = [xx for xx in x if xx[1] < 1.1]

    pprint(x_good)
    x_tasks = [i[0][0] for i in x_good]
    pprint(x_tasks)
    tasks = []
    if len(x_tasks) > 2:
        for i in range(3): 
            tasks.extend([str(t.id) for t in Task.select().where(Task.task_type == x_tasks[i]).order_by(fn.Random()).limit(4)])
    elif 1 < len(x_tasks) < 3: 
        tasks.extend([str(t.id) for t in Task.select().where(Task.task_type == x_tasks[0]).order_by(fn.Random()).limit(4)])
        tasks.extend([str(t.id) for t in Task.select().where(Task.task_type == x_tasks[1]).order_by(fn.Random()).limit(4)])
        tasks.extend([str(t.id) for t in Task.select().where(Task.task_type == x_tasks[0]).order_by(fn.Random()).limit(4)])
    else:
        tasks = [str(t.id) for t in Task.select().where(Task.task_type == x_tasks[0]).order_by(fn.Random()).limit(9)]
    pprint(tasks)
    with open("./data/assignments/ours/{0}.tasks".format(w),'w') as f:
        f.write("#{0}".format(w))
        f.write("\n")
        f.write("{")
        f.write(",".join(tasks[0:3]))
        f.write("}")
        f.write("\n")
        f.write("{")
        f.write(",".join(tasks[3:6]))
        f.write("}")
        f.write("\n")
        f.write("{")
        f.write(",".join(tasks[6:]))
        f.write("}")
    

A10U8R4IEOHQUA
[((6, 0.05), 0.42264973081037427), ((4, 0.04), 0.5285954792089683)]
[6, 4]
['10499',
 '10654',
 '10392',
 '11297',
 '6179',
 '6373',
 '7265',
 '6714',
 '10567',
 '11892',
 '10674',
 '11092']
A11LSO6D7BMY99
[((4, 0.04), 0.6464466094067263)]
[4]
['7237', '7140', '7676', '7669', '7175', '6599', '7697', '7755', '6148']
A11UG30LL53IHA
[((1, 0.02), 0.5527864045000421),
 ((6, 0.05), 0.5527864045000421),
 ((9, 0.05), 0.5527864045000421),
 ((10, 0.02), 0.5527864045000421)]
[1, 6, 9, 10]
['1890',
 '1051',
 '1873',
 '1356',
 '10906',
 '10755',
 '11258',
 '10938',
 '16880',
 '17407',
 '16478',
 '16584']
A128VDS8TMHBDP
[((4, 0.04), 0.36754446796632423)]
[4]
['7828', '6077', '6925', '7075', '7584', '7573', '7729', '6571', '6404']
A12MSYRHI3XWDT
[((6, 0.05), 0.4777670321329065), ((4, 0.04), 0.5735985672887791)]
[6, 4]
['11763',
 '10909',
 '11072',
 '10520',
 '6263',
 '7231',
 '6977',
 '7877',
 '10317',
 '10395',
 '11293',
 '10337']
A12O9P1OTMJOPW
[((1, 0.02), 0.5527864045000421),
 ((6,

[((1, 0.02), 0.5799159747915971),
 ((6, 0.05), 0.5799159747915971),
 ((7, 0.025), 0.5799159747915971),
 ((9, 0.05), 0.5799159747915971),
 ((10, 0.02), 0.5799159747915971)]
[1, 6, 7, 9, 10]
['975',
 '72',
 '1188',
 '1269',
 '11481',
 '11713',
 '10474',
 '10273',
 '12773',
 '12762',
 '13053',
 '13246']
A1RNQ5X21BF816
[((3, 0.03), 0.5), ((10, 0.02), 0.591751709536137)]
[3, 10]
['4693',
 '5563',
 '5546',
 '5886',
 '19402',
 '18294',
 '18223',
 '18629',
 '5022',
 '5047',
 '5744',
 '5532']
A1RWE3R0JP41PC
[((1, 0.02), 0.5669872981077807),
 ((6, 0.05), 0.5669872981077807),
 ((7, 0.025), 0.5669872981077807),
 ((9, 0.05), 0.5669872981077807),
 ((10, 0.02), 0.5669872981077807)]
[1, 6, 7, 9, 10]
['519',
 '915',
 '409',
 '142',
 '11637',
 '11414',
 '11246',
 '10462',
 '12501',
 '12817',
 '13125',
 '12734']
A1SFBJ5WW2OZTI
[((4, 0.04), 0.5),
 ((9, 0.05), 0.591751709536137),
 ((10, 0.02), 0.591751709536137)]
[4, 9, 10]
['6085',
 '6251',
 '6103',
 '6005',
 '16551',
 '17582',
 '16639',
 '16768',
 '18982

A2KHTVC22XK64X
[((1, 0.02), 0.5669872981077807),
 ((6, 0.05), 0.5669872981077807),
 ((7, 0.025), 0.5669872981077807),
 ((9, 0.05), 0.5669872981077807)]
[1, 6, 7, 9]
['1332',
 '1412',
 '221',
 '1935',
 '10205',
 '11809',
 '11614',
 '10368',
 '13923',
 '12618',
 '13906',
 '13530']
A2KKUKAW8EWDAO
[((3, 0.03), 0.5285954792089683)]
[3]
['4771', '5171', '5067', '5389', '4066', '5100', '5360', '5844', '4816']
A2LAE3OM5OQ0WF
[((1, 0.02), 0.5799159747915971),
 ((6, 0.05), 0.5799159747915971),
 ((7, 0.025), 0.5799159747915971),
 ((9, 0.05), 0.5799159747915971),
 ((10, 0.02), 0.5799159747915971)]
[1, 6, 7, 9, 10]
['678',
 '489',
 '637',
 '1086',
 '10160',
 '10437',
 '11452',
 '10746',
 '12918',
 '12760',
 '12752',
 '12292']
A2LBFHF761B64Y
[((4, 0.04), 0.5285954792089683)]
[4]
['7483', '7159', '6187', '7133', '6888', '7902', '6577', '6494', '6379']
A2M0II20JK7R13
[((4, 0.04), 0.5), ((6, 0.05), 0.591751709536137)]
[4, 6]
['6217',
 '6849',
 '7518',
 '7741',
 '11088',
 '11367',
 '11696',
 '11656',
 '

[((10, 0.02), 0.2254033307585166)]
[10]
['19774',
 '19494',
 '18780',
 '19249',
 '18157',
 '19354',
 '18902',
 '18061',
 '18323']
A3HALUPLZ4IKYJ
[((4, 0.04), 0.591751709536137), ((5, 0.02), 0.591751709536137)]
[4, 5]
['7158',
 '6242',
 '7781',
 '7253',
 '8624',
 '9955',
 '8307',
 '9978',
 '6472',
 '7904',
 '7731',
 '7948']
A3HGHX10PBOSMK
[((6, 0.05), 0.42264973081037427)]
[6]
['11324',
 '11826',
 '11431',
 '11377',
 '11579',
 '11069',
 '10376',
 '11381',
 '10319']
A3HSYD08BL5LE2
[((6, 0.05), 0.42264973081037427)]
[6]
['11575',
 '11897',
 '10198',
 '10431',
 '11733',
 '11772',
 '11097',
 '11816',
 '11379']
A3IC7ZQ257WUNV
[((3, 0.03), 0.42264973081037427), ((4, 0.04), 0.42264973081037427)]
[3, 4]
['4612',
 '5178',
 '4227',
 '5707',
 '6209',
 '7529',
 '7321',
 '6214',
 '4751',
 '5758',
 '4401',
 '4885']
A3INON0S827LKQ
[((1, 0.02), 0.5196155385847386),
 ((7, 0.025), 0.5196155385847386),
 ((9, 0.05), 0.5196155385847386)]
[1, 7, 9]
['1377',
 '623',
 '412',
 '1505',
 '12474',
 '12814',
 '1231

[((5, 0.02), 0.36754446796632423)]
[5]
['8247', '9493', '8078', '9247', '8729', '8718', '8360', '9221', '8785']
AMEBYQ08FA7H8
[((10, 0.02), 0.3453463292920229)]
[10]
['19845',
 '18385',
 '18963',
 '18942',
 '18404',
 '19870',
 '18065',
 '18206',
 '19111']
AMJUTNVZ4W26Q
[((2, 0.03), 0.6837722339831621),
 ((4, 0.04), 0.6837722339831621),
 ((5, 0.02), 0.6837722339831621)]
[2, 4, 5]
['3532',
 '2097',
 '2599',
 '3791',
 '6465',
 '6213',
 '7898',
 '6107',
 '9262',
 '9453',
 '9038',
 '9314']
ANWR7DTA3OLLN
[((2, 0.03), 0.5527864045000421), ((3, 0.03), 0.5527864045000421)]
[2, 3]
['3077',
 '2356',
 '3596',
 '3814',
 '5188',
 '4206',
 '4266',
 '5921',
 '3606',
 '2083',
 '3486',
 '2162']
AQ3KPKZ6UIYPF
[((6, 0.05), 0.33333333333333337), ((7, 0.025), 0.33333333333333337)]
[6, 7]
['10337',
 '10190',
 '10013',
 '10689',
 '13156',
 '12154',
 '12029',
 '13954',
 '10864',
 '10866',
 '11380',
 '11669']
AQPWA0ZDAGHGM
[((6, 0.05), 0.5285954792089683)]
[6]
['10549',
 '11917',
 '10530',
 '11295',
 '10662',
 